In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("StockBasic")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

22/03/04 03:31:29 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/04 03:31:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/04 03:31:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/04 03:31:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/04 03:31:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/03/04 03:31:31 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
SectorsDf = spark.read.format('csv')\
                    .option('header', True)\
                    .option('inferSchema', True)\
                    .load("hdfs://localhost:9000/stocks/sectors")

dailyDf = spark.read.format('csv')\
                    .option('header', True)\
                    .option('inferSchema', True)\
                    .load("hdfs://localhost:9000/stocks/daily")

# to drop column the NAME has to be in double qoutes - "NAME" 
dailyDf = dailyDf.drop("_c13")
SectorsDf.show(5)
dailyDf.show(5)

+--------------------+------------------+----------+------+------------+
|        Company Name|          Industry|    Symbol|Series|   ISIN Code|
+--------------------+------------------+----------+------+------------+
|      Axis Bank Ltd.|FINANCIAL SERVICES|  AXISBANK|    EQ|INE238A01034|
|  Bajaj Finance Ltd.|FINANCIAL SERVICES|BAJFINANCE|    EQ|INE296A01024|
|  Bajaj Finserv Ltd.|FINANCIAL SERVICES|BAJAJFINSV|    EQ|INE918I01018|
|Cholamandalam Inv...|FINANCIAL SERVICES|  CHOLAFIN|    EQ|INE121A01024|
|HDFC Asset Manage...|FINANCIAL SERVICES|   HDFCAMC|    EQ|INE127D01025|
+--------------------+------------------+----------+------+------------+
only showing top 5 rows

+----------+------+-------+-------+-------+-------+-------+---------+---------+-------------+-----------+-----------+------------+
|    SYMBOL|SERIES|   OPEN|   HIGH|    LOW|  CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|
+----------+------+-------+-------+-------+-------+-----

In [4]:
# add/derive a column withColumn Gain = CLOSE - OPEN
# add/derive a column withColumn GAINP = (abs(GAIN) / OPEN)
# where as abs is function from pyspark.sql.functions
from pyspark.sql.functions import col
dailyDf = dailyDf.withColumn("GAIN", col("CLOSE") - col("OPEN"))\
                 .withColumn("GAINP", (col("GAIN") / col("OPEN")) * 100)

dailyDf.printSchema()
dailyDf.show(5)

root
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- LAST: double (nullable = true)
 |-- PREVCLOSE: double (nullable = true)
 |-- TOTTRDQTY: integer (nullable = true)
 |-- TOTTRDVAL: double (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: integer (nullable = true)
 |-- ISIN: string (nullable = true)
 |-- GAIN: double (nullable = true)
 |-- GAINP: double (nullable = true)

+----------+------+-------+-------+-------+-------+-------+---------+---------+-------------+-----------+-----------+------------+--------------------+-------------------+
|    SYMBOL|SERIES|   OPEN|   HIGH|    LOW|  CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|                GAIN|              GAINP|
+----------+------+-------+-------+-------+-------+-------+---------+-------